In [41]:
# Only run this cell on Colab!
#!pip install control==0.9.0 import-ipynb==0.1.3 joblib==1.1.0 simpleaudio==1.0.4 soundfile==0.10.3.post1 threadpoolctl==3.0.0 &> /dev/null

# if you run this locally on Jupyter notebook, run the following command in your Anaconda terminal (without the #)
# pip install control==0.9.0 import-ipynb==0.1.3 joblib==1.1.0 simpleaudio==1.0.4 soundfile==0.10.3.post1 threadpoolctl==3.0.0

In [42]:

import soundfile as sf
import scipy.signal as ss
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
from audio_read import audio_read
from mux import mux
from demux import demux
from audio_save import audio_save
from play_buffer_wrapper import play_buffer_wrapper
from upconverter import upconverter
from downconverter import downconverter


# Exercise 2

In telecommunication systems, multiplexing is a technique that allows multiple signals to be transmitted simultaneously, avoiding interference among different channels. One method for multiplexing is Frequency Division Multiplexing (FDM), which will be analyzed in this exercise.

Before diving into how it works, a couple of definitions are necessary:

**Baseband bandlimited** : the DFT of the signal $x[n]$ with bandwidth $f_{bw}$ is $X(f) = 0, \ \forall |{f}|>f_{bw}$. 

**Passband** : the energy of the signal $x[n]$ is distributed through the frequencies $f\in[f_1,f_2]$ where $f_1<f_2,\ f_1> 0$ and $f_2<f_s/2$. In other words $\forall f\notin[f_1,f_2]$ we have $X(f)=0$. Such a passband signal has bandwidth $f_{bw}=(f_2-f_1)/2$

Please note that in this homework we work with DFT coefficients expressed as frequencies $f$ in Hertz (similar to Homework 3). The discrete-time angular frequency $\Omega_k = \frac{2\pi k}{N}$ is associated with the continuous-time frequency $f$ via the following relationships: $f = \frac{\Omega_k}{2\pi T_s} = \frac{\Omega_k}{2\pi}f_s$ and $\Omega_k = 2\pi f T_s = 2\pi \frac{f}{f_s}$. Alternatively, you can also use the relationship $f = \frac{k}{NT_s}$, where $k$ is the index of $\Omega_k$.

<b>Task 1.</b> 

The idea of multiplexing is to simultaneously transmit $N$ baseband bandlimited signals, $\{y_i[n]\}_{i\in[N]}$ with bandwidth $f_{bw}$ without interference by exploiting the orthogonality of non-overlapping frequency bands. The following steps are carried out to achieve this: 

1. Each of the N signals is first *upconverted* to a carrier frequency $f_{c_i}$. *Upconversion* is the operation that shifts a signal to the right in the frequency domain, in this case from a baseband to a passband signal. Mathematically, let $Y_i(f)$ be the DFT of the baseband signal $y_i[n]$ and $Y_i^u(f)$ the DFT of the baseband signal upconverted to the carrier frequency $f_{c_i}$. Then the following relationship holds: $Y_i^u(f) = Y_i(f-f_{c_i})$. We exploit this property for every $\{y_i[n]\}_{i\in[N]}$.

2. The resulting upconverted signal $y^u_i[n]$ is  passband and complex. Since we cannot transmit imaginary signals, usually the real part of the upconverted signal is kept, giving $\hat{y}_i[n] = \alpha \operatorname{Re}\{y^u_i[n]\}$. The coefficient $\alpha$ ensures that the energies of $\hat{y}_i$ and $y_i$ remain the same.

3. The last step of FDM is to sum up the $\{\hat{y}_i[n]\}_{i\in[N]}$ signals and transmit the sum.

    
<b>(a)</b> Why are audio files usually sampled at 44.1kHz? For a signal of bandwidth $f_{bw}$ and a sample rate $f_s$, what is the maximum carrier frequency to which one could upconvert? What happens if you go beyond this carrier frequency? 

<b>(b)</b> Define the mathematical relationship, in the time domain, between the baseband signal $y_i[n]$ and the real passband signal $\hat{y}_i[n]$. Refer to the upconversion description in the introduction.

*Hint* : Start from $Y_i^u(f) = Y_i(f-f_{c_i})$ and derive an equivalent expression in the time domain

<b>(c)</b> Using your analytical result from (b), implement the **upconverter** function. Your function takes as an input argument the length of the signal which will be upconverted, i.e. the length of $y_i[n]$. The output of this function can be directly multiplied with the time domain audio signal to obtain the upconverted signal $y^u_i[n]$.

*Note*: Use the code in the following cell to test your implementation. You don't need to modify it.

In [43]:

# Read audio file
audio, Fs= audio_read('audio/original/champions.wav')
index= 0
L= len(audio)

# Bandwidth frequency
F_bw= 3.7e3

# Upconversion
up= upconverter(index, L, Fs, F_bw)
y_up= (up*audio)
y_up.shape[0]


661871

<b>(d)</b> Using your **upconverter**, implement the **mux** function, with carrier frequency $f_{c_i} = (2i+1)f_{mux,bw}$ and $f_{mux,bw} = 3.7$ kHz. We denote $i=0,1,2...$ as the index of the audio file the upconverter is for.

*Hint*: First, compute a different upconverter for each audio file $i$, then use your upconverters to compute $y^u_i[n]$ and finally sum over all of your indexes (step 3 for the FDM).

*Note*: Use the code in the following cell to test your implementation. Do not modify it. 

In [44]:

# Reading audio files
audio1, Fs = audio_read('audio/original/champions.wav')                   #Audio file 1
audio2, Fs = audio_read('audio/original/iphone.wav')                      #Audio file 2
audio3, Fs = audio_read('audio/original/the_polish_ambassador.wav')       #Audio file 3

L= len(audio3)                                                            # length of audio file 3

F_bw = 3.7e3                                                              # Bandwidth frequency

max_len = max(len(audio1), len(audio2), len(audio3))                      # Selecting file with the highest length
y1_padded = np.pad(audio1, (0, max_len - len(audio1)))
y2_padded = np.pad(audio2, (0, max_len - len(audio2)))
y3_padded = np.pad(audio3, (0, max_len - len(audio3)))

y_list = [y1_padded, y2_padded, y3_padded]                                # creating list of audio signals
mux_signal = mux(y_list, Fs, F_bw,L)                                      # Multiplexing of signals Proceeds


<b>(e)</b> Using **mux**, read the 3 audio files provided in *audio/original/* and generate a multiplexed **mux_signal** and save it as *mux.wav*. 

*Hint* : This is just an extension of the code provided in the previous cell. Use the provided functions **audio_read**, **audio_save** and **play_buffer_wrapper**.

*Note* : We do not provide you a code template for this question, implement your solution in the cells below.

In [45]:
#muxing
mux_signal /= np.max(np.abs(mux_signal))
audio_save('saved_track/mux1_1.wav', mux_signal, Fs)   # saving final multiplexed signal as mux1_1.wav file in saved_track folder

<b>Task 2.</b> 

Demultiplexing is the inverse operation of multiplexing. Given a multiplexed signal, this operation restores the $N$ original baseband signals. This is done by *downconverting* the multiplexed signal $N$ times with carrier frequencies $\{f_{c_i}\}_{i\in[N]}$. *Downconversion*, like upconversion, also shifts a signal in the frequency domain, except that it shifts to the left instead of to the right. Then, these $N$ downconverted signals are each passed through a lowpass filter with cutoff frequency $f_{p} = f_{mux,bw} =  f_{c_1}$ (called the multiplexing bandwidth) to obtain the $N$ original signals. Keep in mind that the energies of the resulting signals should be the same as the original, so there will be a normalizing coefficient here too. In terms of real world applications: most wireless communication systems use this technique to communicate without interference. As a result one cannot transmit at any randomly chosen carrier frequency as this will possibly interfere with another transmission. Each country has a frequency allocation plan that must be respected. If you look at one of these frequency allocation plans, it is impressive to see how many different communication systems are transmitting at the same time and to consider that this technique allows them not to interfere with one another.

<b>(a)</b> Rewrite the relationship that you derived in Task 1 (b) and implement the **downconverter** function that reverses the *Upconversion*.

*Note*: Use the code in the following cell to test your implementation. You don't need to modify it.

In [46]:

# Reading audio file
audio, Fs = audio_read('audio/original/champions.wav')
L=len(audio)

# Bandwidth frequency
F_bw=3.7e3

# Column of the desired audio channel
index=0

# Downconversion using the downconverter function
down= downconverter(index, L, Fs, F_bw)


<b>(b)</b> Using your **downconverter**, implement the **demux** function that is able to restore the original audio signals contained in the mux. Your function takes the number of original audio signals in the mux as an input argument **audio_col**. This needs to be known beforehand.

*Hint*: Similar to the **mux** function, you can apply your **demux** to the **mux_signal** in order to obtain the downconverted signals. Don't forget to filter the downconverted signals. You want to pass the $N$ downconverted signals through a lowpass filter with cutoff frequency $f_{p} = f_{mux,bw} =  f_{c_1}$ to obtain the $N$ original signals (use one of the Python native filters to do that). The output **demux_signal** should have one column for each of the original audio signals. 

*Note* : We do not provide you a code template for this question, implement your solution in the cells below.

In [47]:
#demuxing phase

demux_filtered = demux(mux_signal,Fs,3,F_bw,1,2)
demux_filtered

#printing to show value of the bandwith frequency, sampling frquency and the demuxed filtered
print(f'{F_bw}')
print(f'{Fs}')
print(f'{demux_filtered}')



3700.0
44100
[[-0.49497157 -0.49490078 -0.49144634]
 [-0.31365313 -0.2968559  -0.39715407]
 [-0.16479348 -0.13570538 -0.32119604]
 ...
 [-0.42953236 -0.2843778  -1.03071525]
 [-0.82642925 -0.67693385 -0.90373513]
 [-1.30837347 -1.14297509 -0.79794595]]


<b>(c)</b> Using your **demux**, you can demultiplex the previously generated *mux.wav* audio file. You should be able to restore audio signals that are almost like the original audio files. Save the resulting audio file as *demux_i.wav*, where i corresponds to the index of the restored audio.

*Hint* : Use the DFT to determine $f_{bw}$, the bandwidth, $N$, the number of multiplexed signals, and the different $\{f_{c_i}\}_{i\in[N]}$.

*Note* : We do not provide you a code template for this question, implement your solution in the cells below. If you did not succeed in implementing the filtering functions, you may use the native Python functions. If you did not manage to generate the *mux.wav* in the previous task you may use the *mux_sample.wav*, which contains 5 audio signals. 

In [48]:
Song1_1 = demux_filtered[0:, 0]
Song2_2 = demux_filtered[0:, -2]
Song3_3 = demux_filtered[0:, -1]

song1_1 = Song1_1[:661872]
song2_2 = Song2_2[:661872]

#song1_1, song2_2 and song3_3 saved as demux1_1, demux2_2 and demux3_3 in the saved_track folder
song1 = audio_save('saved_track/demux1_1.wav', song1_1, Fs)      
song2 = audio_save('saved_track/demux2_2.wav', song2_2, Fs)
song3  =audio_save('saved_track/demux3_3.wav', Song3_3, Fs)